# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

**Цель исследования:** обучить модель классифицировать комментарии на позитивные и негативные, значение метрики качества *F1* обученной модели  должна быть не меньше 0.75. 

**Ход исследования:**

1. *Подготовка данных:*

- Открыть файл и загрузить данные с использованием библиотеки pandas;
- Провести первичную оценку данных: использовать методы head(), info() для изучения общей информации;
- Провести лемматизацию и очистку данных.

2. *Обучение:*

- Избавиться от дисбаланса классов;
- Построить пайплайн;
- Оценить метрику F1 на тестовой выборке для лучше модели.

*Общий вывод:*

- Написать общий вывод, в котором суммировать основные результаты исследования и сделать ключевые выводы.

In [1]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
import re 
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
try:
    df = pd.read_csv("/datasets/toxic_comments.csv")
except FileNotFoundError:
    df = pd.read_csv(r"C:\Users\Тадевос\Tadevos\USER\Desktop\Курсы\Модуль 4\Датасеты к проектам\МО для текстов\toxic_comments.csv")

In [3]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


## 1. Подготовка данных

In [5]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Тадевос\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Тадевос\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Тадевос\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
corpus = df['text'].values

# Создание экземпляра лемматизатора WordNet
lemmatizer = WordNetLemmatizer()

# Функция лемматизации текста с указанием части речи
def lemmatize(text):
    words = word_tokenize(text)  # Токенизация текста
    lemmatized_words = [lemmatizer.lemmatize(word, pos='v') for word in words]  # Лемматизация с указанием части речи (глагол)
    lemmatized_text = ' '.join(lemmatized_words)
    return lemmatized_text

# Функция очистки текста от спецсимволов
def clear_text(text):
    no_special_chars_text = re.sub(r'[^a-zA-Z]', ' ', text)  # Удаление спецсимволов, оставляем только латинские буквы
    clear_text = " ".join(no_special_chars_text.split()) 
    return clear_text

# Применение функции к тексту и добавление столбца clean_text в DataFrame
df['lemm_text'] = df['text'].apply(lambda x: lemmatize(clear_text(x)))

# Приведение текста в столбце 'lemm_text' к нижнему регистру
df['lemm_text'] = df['lemm_text'].str.lower()

print("Исходный текст:", corpus[0])
print("Очищенный и лемматизированный текст:", lemmatize(clear_text(corpus[0])))

Исходный текст: Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27
Очищенный и лемматизированный текст: Explanation Why the edit make under my username Hardcore Metallica Fan be revert They weren t vandalisms just closure on some GAs after I vote at New York Dolls FAC And please don t remove the template from the talk page since I m retire now


In [7]:
df.head()

,Unnamed: 0,text,toxic,lemm_text
0,0,Explanation\nWhy the edits made under my usern...,0,explanation why the edit make under my usernam...
1,1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not try to edit war it s ju...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestions on impr...
4,4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


> **Выводы по первому шагу:**
>
> В первом шаге были проделаны следующие действия:
>  - загрузка данных;
>  - изучение общей информации о данных.
>
> **После загрузки и изучения данных можно сделать несколько выводов:**
> 1. лемматизирован и чищен текст;
> 2. добавлен в датафрейм столбец с очищенным и лемматизированным текстом.

## 2. Обучение
### 2.1 Избавимся от дисбаланса классов и разделим данные

In [8]:
df['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [12]:
RANDOM_STATE = 42
TEST_SIZE = 0.25

condensed_df = df.sample(50000, random_state=RANDOM_STATE).reset_index(drop=True)

# Assuming 'df' is your DataFrame
X = condensed_df['lemm_text']
y = condensed_df['toxic']

# Разделение данных на обучающую и тестовую выборки с использованием стратификации
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, 
                                                    stratify=y, random_state=RANDOM_STATE)

### 2.2 Построим пайплайн

In [16]:
# List of models with their corresponding parameter grids
models = [
    ('Logistic Regression', LogisticRegression(class_weight='balanced'), {
        'clf__C': [0.1, 1, 10]
    }),
    ('LinearSVC', LinearSVC(), {
        'clf__C': [0.01, 0.1, 1, 10, 100],
        'clf__penalty': ['l1', 'l2']
    }),
    ('LightLGBM', LGBMClassifier(), {
        'clf__num_leaves': [10, 20, 30],
        'clf__max_depth': [5, 10, -1]
    })
]

# Training and evaluation of models
results = []

for model_name, model, param_grid in models:
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', model)
    ])
    
    start_train = time.time()
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    end_train = time.time()

    # Получаем лучшую модель и её параметры
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    
    # Оценка времени предсказания на обучающей выборке
    start_pred_train = time.time()
    y_pred_train = best_model.predict(X_train)
    end_pred_train = time.time()

    # Рассчитываем время обучения и время предсказания
    train_time = (end_train - start_train) / 60
    pred_train_time = (end_pred_train - start_pred_train) / 60

    # Сохраняем результаты
    results.append({
        'Model': model_name,
        'Best Params': best_params,
        'Best Score': round(grid_search.best_score_, 2),
        'Training Time (s)': train_time,
        'Prediction Time Train (s)': pred_train_time
    })

    # Выводим результаты для обучающей выборки
    print(f'Модель: {type(model).__name__}')
    print(f'Лучшие параметры: {best_params}')
    print(f'Метрика лучшей модели на обучающей выборке: {round(grid_search.best_score_, 2)}')

Модель: LogisticRegression
Лучшие параметры: {'clf__C': 10}
Метрика лучшей модели на обучающей выборке: 0.75
Модель: LinearSVC
Лучшие параметры: {'clf__C': 1, 'clf__penalty': 'l2'}
Метрика лучшей модели на обучающей выборке: 0.76
Модель: LGBMClassifier
Лучшие параметры: {'clf__max_depth': -1, 'clf__num_leaves': 30}
Метрика лучшей модели на обучающей выборке: 0.72


In [17]:
# Создаем DataFrame из списка результатов
results_df = pd.DataFrame(results)

# Выводим таблицу с результатами
results_df

,Model,Best Params,Best Score,Training Time (s),Prediction Time Train (s)
0,Logistic Regression,{'clf__C': 10},0.75,0.150956,0.018961
1,LinearSVC,"{'clf__C': 1, 'clf__penalty': 'l2'}",0.76,0.315092,0.019082
2,LightLGBM,"{'clf__max_depth': -1, 'clf__num_leaves': 30}",0.72,1.658257,0.021499


### 2.4 Оценим метрику лучшей модели на тестовой выборке

In [18]:
# Получаем лучшие параметры
best_params = grid_search.best_params_

# Создаем пайплайн с лучшей моделью и ее параметрами
best_model = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LinearSVC(C=1, penalty='l2'))
])

# Обучаем лучшую модель на всем обучающем наборе данных
best_model.fit(X_train, y_train)

# Оценка качества на тестовой выборке
y_pred_test = best_model.predict(X_test)

# Расчет F1-меры на тестовой выборке
f1_test = f1_score(y_test, y_pred_test)

# Выводим результат для тестовой выборки
print(f'Лучшая модель и её параметры: {best_model}')
print(f'F1-мера для лучшей модели на тестовой выборке: {f1_test:.2f}\n')

Лучшая модель и её параметры: Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC(C=1))])
F1-мера для лучшей модели на тестовой выборке: 0.77



> **Выводы по второму шагу:**
>
> Во втором шаге были проделаны следующие действия:
>  - дисбаланс классов;
>  - построен пайплайн.
>
> **После проведённых действий можно сделать следующие выводы:**
> 1. избавились от дисбаланса классов;
> 2. обучены 3 модели: **LogisticRegression**, **LinearSVC**, **LGBMClassifier**;
> 3. оценена метрика лучшей модели **LinearSVC** на тестовой выборке, по условию значение метрики F1 > 0.75.

## 3. Общий вывод

В ходе исследования был проведен анализ данных для определения оптимального инструмента, способного автоматически обнаруживать токсичные комментарии среди предложенных покупателями правок и комментариев. Для этого были выполнены следующие шаги:

**Подготовка данных:** Исходные данные были загружены и предварительно обработаны для дальнейшего анализа.

**Выбор моделей:** Был составлен список моделей машинного обучения, которые могли бы быть применены для решения задачи обнаружения токсичных комментариев.

**Настройка конвейера:** Для каждой модели был создан конвейер (pipeline), который включал в себя предварительную обработку текста (например, TF-IDF векторизацию) и обучение модели.

**Подбор гиперпараметров:** Для каждой модели был проведен поиск по сетке для определения наилучших гиперпараметров с использованием кросс-валидации на обучающем наборе данных.

**Обучение моделей:** Наилучшие модели были обучены на обучающем наборе данных с использованием найденных лучших гиперпараметров.

**Оценка качества моделей:** Качество каждой модели было оценено с использованием F1-меры на обучающем наборе данных.

В ходе исследования было выявлено, что модель LinearSVC показала наилучшее качество среди рассмотренных моделей. Она была выбрана в качестве наилучшей модели для решения задачи обнаружения токсичных комментариев.

Для данной модели были подобраны оптимальные гиперпараметры, включая параметры регуляризации (C) и тип регуляризации (penalty). Обученная модель была успешно протестирована на тестовом наборе данных, и ее качество было оценено с использованием F1-меры, которая составила около 0.77.

Таким образом, модель LinearSVC с настроенными гиперпараметрами является оптимальным инструментом для автоматического обнаружения токсичных комментариев в интернет-магазине «Викишоп».

### Чек-лист готовности проекта

- [x]  импортированы все необходимые библиотеки.

**Шаг 1. Загрузка данных**
- [x]  загружены данные;
- [x]  изучена общая информация;
- [x]  написан вывод.

**Шаг 2. Обучение**
- [x]  дисбаланс классов;
- [x]  построен пайплайн;
- [x]  написан вывод.

**Общий вывод**
- [x]  написан общий вывод;